In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from tqdm import tqdm
import numpy as np
import pandas as pd
import os
import shutil
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
ivscc_shiny = pd.read_csv("../inh_ivscc_shiny_ttype_and_quality.csv")
ivscc_met_types = pd.read_csv('../20200711_patchseq_metadata_mouse.csv')[['cell_specimen_id','MET-type Label']]

ivscc_labels = ivscc_shiny.merge(ivscc_met_types,left_on='spec_id_label',right_on='cell_specimen_id')
ivscc_labels = ivscc_labels[ivscc_labels['Tree_call_label']!="PoorQ"]

ivscc_features = pd.read_csv("../IVSCC_Features_correct_layer_template/RawFeatureWide.csv")
em_features = pd.read_csv("./SkKeyFeatures_11_15_2022/RawFeatureWide.csv")

ivscc_merged = ivscc_features.merge(ivscc_labels,left_on='specimen_id',right_on='spec_id_label')

In [9]:
feature_columns = ivscc_features.columns[1:]
catch_flags = ["area","vol","diam"]
feature_columns = [c for c in feature_columns.tolist() if not any([catch in c for catch in catch_flags])]
feature_columns

['axon_bias_x',
 'axon_bias_y',
 'axon_depth_pc_0',
 'axon_depth_pc_1',
 'axon_depth_pc_2',
 'axon_depth_pc_3',
 'axon_depth_pc_4',
 'axon_emd_with_basal_dendrite',
 'axon_exit_distance',
 'axon_exit_theta',
 'axon_extent_x',
 'axon_extent_y',
 'axon_frac_above_basal_dendrite',
 'axon_frac_below_basal_dendrite',
 'axon_frac_intersect_basal_dendrite',
 'axon_max_branch_order',
 'axon_max_euclidean_distance',
 'axon_max_path_distance',
 'axon_mean_contraction',
 'axon_num_branches',
 'axon_soma_percentile_x',
 'axon_soma_percentile_y',
 'axon_total_length',
 'basal_dendrite_bias_x',
 'basal_dendrite_bias_y',
 'basal_dendrite_calculate_number_of_stems',
 'basal_dendrite_extent_x',
 'basal_dendrite_extent_y',
 'basal_dendrite_frac_above_axon',
 'basal_dendrite_frac_below_axon',
 'basal_dendrite_frac_intersect_axon',
 'basal_dendrite_max_branch_order',
 'basal_dendrite_max_euclidean_distance',
 'basal_dendrite_max_path_distance',
 'basal_dendrite_mean_contraction',
 'basal_dendrite_num_bran

In [14]:
rfc_met_result_df, rfc_met_results_verbose, all_oob_accuracies = predict_labels_with_probability(source_dataframe=ivscc_merged,
                                                                                                prediction_dataframe=em_features,
                                                                                                prediction_id_column='specimen_id',
                                                                                                feature_columns=feature_columns,
                                                                                                label_column='MET-type Label',
                                                                                                classifier_method="RFC",
                                                                                                min_class_size = 5,
                                                                                                min_class_size_subsampling=5,
                                                                                                feature_weights=None,
                                                                                                num_iterations=500,
                                                                                                subsampling_rate=0.95,
                                                                                               )

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [05:41<00:00,  1.46it/s]


In [16]:
np.mean(all_oob_accuracies), np.std(all_oob_accuracies)

(0.5800044052863437, 0.012695007249574107)

In [19]:
rfc_met_result_df.to_csv("EM_MC_Predicted_MET_Labels_11_18_22.csv")
# rfc_met_results_verbose.to_csv("EM_MC_Predicted_MET_Labels_11_18_22_verbose_raw_cts.csv")

In [27]:
old_results = pd.read_csv("../big_batch_v1/EM_MC_Predicted_MET_Labels_10_3_22.csv",index_col=0)
old_results = old_results.rename(columns={'predicted_MET-type Label':'OLD_pMET','probability':'old_probability'})

In [35]:
merged = rfc_met_result_df.merge(old_results,on='id')
merged[merged['predicted_MET-type Label']==merged['OLD_pMET']]

,id,predicted_MET-type Label,probability,OLD_pMET,old_probability
0,864691135013417622,Sst-MET-4,1.000,Sst-MET-4,0.990476
1,864691135058985115,Sst-MET-6,0.994,Sst-MET-6,0.971429
2,864691135118298333,Sst-MET-6,0.598,Sst-MET-6,0.647619
3,864691135292179382,Sst-MET-4,0.846,Sst-MET-4,0.714286
5,864691135341516741,Sst-MET-9,0.884,Sst-MET-9,0.704762
6,864691135374222153,Sst-MET-6,0.996,Sst-MET-6,0.980952
7,864691135446065810,Sst-MET-4,1.000,Sst-MET-4,0.990476
8,864691135467660940,Sst-MET-4,1.000,Sst-MET-4,1.000000
9,864691135544588584,Sst-MET-8,0.998,Sst-MET-8,1.000000
10,864691135577202181,Sst-MET-4,0.998,Sst-MET-4,0.961905


In [11]:
def predict_labels_with_probability(source_dataframe,
                                    prediction_dataframe,
                                    prediction_id_column,
                                    feature_columns,
                                    label_column,
                                    classifier_method,
                                    min_class_size = 5,
                                    min_class_size_subsampling=5,
                                    feature_weights=None,
                                    num_iterations=500,
                                    subsampling_rate=0.95,
                                   ):
    
    """
    Given morpho feature data collected from patch-seq (IVSCC) and EM, we will predict the MET cell type labels
    from patch-seq to the EM cells. 


    :param source_dataframe: dataframe, the training dataframe (IVSCC) must have all feature_columns and label_column (e.g. met_type)
    :param prediction_dataframe: dataframe, the dataframe you would like to assign labels to, must have all feature columns and an id column
    :param prediction_id_column: str, specimen id column in the prediction dataframe
    :param feature_columns: list of features, must be present in both dataframes
    :param label_column: str, what column from the source_dataframe  you want to predict 
    :param classifier_method: str, [KNN or RFC]
    :param min_class_size: int, the smallest representation allowed in training data, instances from a class with fewer than this value will be dropped
    :param min_class_size_subsampling: int, when subsampling, what is the smallest you want to erode any given class of training data
    :param feature_weights: list/array of feature weights if you want to scale them for any distance based metrics
    :param num_iterations: int, how many iterations of classification to run
    :param subsampling_rate: float in range 0-1.0. Represents the percent of IVSCC cells to subsample in each iteration


    :return: result_df: dataframe, has the following columns: ["id",'predicted_label_column','probability']
    :return: results: mxn numpy array, m = prediction cells, n=all unique possible labels, the ith,jth value indicates how 
    many times the ith cell was predited to be the jth label
    :return: all_accuracies: list, keeps track of either oob_score or training error of each classifier
    """

    prediction_dataframe.fillna(0,inplace=True)
    
    source_dataframe=source_dataframe.copy()
    
    labels_above_threshold = [k for k,v in source_dataframe[label_column].value_counts().to_dict().items() if v>=min_class_size]
    source_dataframe = source_dataframe[source_dataframe[label_column].isin(labels_above_threshold)]
    
    num_source_datapoints_to_drop = int(len(source_dataframe)*(1-subsampling_rate))

    if not feature_weights:
        feature_weights = [1]*len(feature_columns)
        
    prediction_data_array = prediction_dataframe[feature_columns].values
    prediction_data_array = np.multiply(prediction_data_array,feature_weights)
    prediction_ids = prediction_dataframe[prediction_id_column].tolist()
    
    unique_labels = source_dataframe[label_column].unique().tolist()
    results = np.zeros((len(prediction_ids), len(unique_labels)))
    

    all_accuracies = []
    for outter_it in tqdm(range(num_iterations)):
        #subsample the entirety of our source dataset according to subsampling_rate
        source_labels_value_counts = source_dataframe[label_column].value_counts().to_dict()
        source_dataframe_subsampled = source_dataframe.copy(deep=True)
        for _ in range(num_source_datapoints_to_drop):

            available_labels_to_drop = [k for k,v in source_labels_value_counts.items() if v>min_class_size_subsampling]
            current_number_of_cells = sum([source_labels_value_counts[v] for v in available_labels_to_drop])
            probability_of_available_labels = [source_labels_value_counts[k]/current_number_of_cells for k in available_labels_to_drop]
            if available_labels_to_drop:
                label_to_drop = np.random.choice(available_labels_to_drop,p=probability_of_available_labels)
                idx_to_drop = np.random.choice(source_dataframe_subsampled[source_dataframe_subsampled[label_column]==label_to_drop].index)
                source_dataframe_subsampled = source_dataframe_subsampled.drop(idx_to_drop)
                source_labels_value_counts[label_to_drop] = source_labels_value_counts[label_to_drop] - 1 

                

        
        feature_values = source_dataframe_subsampled[feature_columns].values
        feature_values = np.multiply(feature_values,feature_weights)
        labels = source_dataframe_subsampled[label_column].values
        
        
        if classifier_method == "RFC":
            classifier = RandomForestClassifier(n_estimators=200,
                                                 random_state=0,
                                                 min_samples_leaf=3,
                                                 min_samples_split=3,
                                                 oob_score=True,
                                                 max_depth = None, 
                                                 class_weight='balanced'
                                                )
            
        elif classifier_method == "KNN":
            classifier = KNeighborsClassifier(n_neighbors=5)
            
        else:
            print("Invalid classifier_method passed")
            return None,None,None
        
        classifier.fit(feature_values, labels)
        
        if hasattr(classifier, 'oob_score'):
            all_accuracies.append(classifier.oob_score_)
            
        else:
            all_accuracies.append(classifier.score(feature_values, labels))

        predictions = classifier.predict(prediction_data_array)
        predictions_by_id = dict(zip(prediction_ids,predictions))
        # keep track of predictions 
        for sample_id, pred_label in predictions_by_id.items():
            row_idx = prediction_ids.index(sample_id)
            col_ids = unique_labels.index(pred_label)
            results[row_idx, col_ids]+=1
        
       
    # create simple dataframe     
    chosen_label_indices = np.argmax(results,axis=1)
    chosen_label_occurence = np.max(results,axis=1)
    chosen_labels = [unique_labels[i] for i in chosen_label_indices]
    probability = chosen_label_occurence/num_iterations

    result_df = pd.DataFrame({"id":prediction_ids,
                 f"predicted_{label_column}":chosen_labels,
                 "probability":probability})

    return result_df, results, all_accuracies

